In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import glob
from skimage.io import imread_collection
import os
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import imageio as iio
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
keras = tf.keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet121
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.merge import concatenate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix, confusion_matrix  

#Load Dataset

In [ ]:
#Create labels for the images. 
ts_labels = []
i = 33
while(i>0):
  ts_labels.append(np.array([0])) #Say all images belong to class 0
  i = i-1

tr_labels = []
i = 329
while(i>0):
  tr_labels.append(np.array([1])) #1 for actual face. 
  i = i-1
i = 261
while(i>0):
  tr_labels.append(np.array([0])) #0 for tree. 
  i = i-1

In [ ]:
print(len(tr_labels), len(ts_labels))

590 33


In [ ]:
tr_labels = np.array(tr_labels)
tr_labels.shape
ts_labels = np.array(ts_labels)

In [ ]:
#Read all the images from dataset. 
ts_images = []
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Face vs Trees - Test Data/*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Face vs Trees - Test Data/*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Face vs Trees - Test Data/*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    ts_images.append(n)

tr_images = []
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Faces Train (Real, Other Objects, Abstract Paintings, Sketches)/*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Faces Train (Real, Other Objects, Abstract Paintings, Sketches)/*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Faces Train (Real, Other Objects, Abstract Paintings, Sketches)/*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Tree Sketches - Train /*.jpeg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Tree Sketches - Train /*.jpg"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)
for img in glob.glob("/content/drive/MyDrive/Data Sets - Colab/UTS Thesis/Face vs Tree/Tree Sketches - Train /*.png"):
    n= cv2.imread(img)
    n = cv2.resize(n, (224, 224))
    tr_images.append(n)

In [ ]:
tr_images = np.array(tr_images)
ts_images = np.array(ts_images)
print(tr_images.shape)
print(ts_images.shape)
print(len(tr_labels))
print(len(ts_labels))
print(type(tr_labels))

(590, 224, 224, 3)
(33, 224, 224, 3)
590
33
<class 'numpy.ndarray'>


In [ ]:
#Data Augmentation

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images upto 180 degrees
        shear_range=0.2,
        zoom_range = 0.15, # randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=False, # randomly flip images vertically
        fill_mode='nearest')  


datagen.fit(tr_images)

#CNN Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 173056)            0

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

hist = model.fit(tr_images, tr_labels, epochs=10, 
                    validation_data=(ts_images, ts_labels))

Epoch 1/10
19/19 [==============================] - 13s 94ms/step - loss: 436.2370 - accuracy: 0.6102 - val_loss: 0.7627 - val_accuracy: 0.3636
Epoch 2/10
19/19 [==============================] - 1s 49ms/step - loss: 0.5613 - accuracy: 0.8068 - val_loss: 2.0609 - val_accuracy: 0.2121
Epoch 3/10
19/19 [==============================] - 1s 46ms/step - loss: 0.4460 - accuracy: 0.8492 - val_loss: 1.1955 - val_accuracy: 0.4545
Epoch 4/10
19/19 [==============================] - 1s 45ms/step - loss: 0.2455 - accuracy: 0.9085 - val_loss: 5.0340 - val_accuracy: 0.0606
Epoch 5/10
19/19 [==============================] - 1s 45ms/step - loss: 0.7534 - accuracy: 0.9068 - val_loss: 2.5909 - val_accuracy: 0.4242
Epoch 6/10
19/19 [==============================] - 1s 46ms/step - loss: 1.6938 - accuracy: 0.8051 - val_loss: 1.3304 - val_accuracy: 0.2121
Epoch 7/10
19/19 [==============================] - 1s 46ms/step - loss: 0.4131 - accuracy: 0.8508 - val_loss: 9.6520 - val_accuracy: 0.0909
Epoch 8/10

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 2.3727 - accuracy: 0.1212 - 34ms/epoch - 17ms/step
0.12121212482452393


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[ -0.13074991   0.11652727]
 [ -0.34311473   0.3093592 ]
 [ -2.1626222    2.1188033 ]
 [ -3.0370421    1.9513865 ]
 [ -3.197718     3.1407797 ]
 [ -1.4052156    1.4491369 ]
 [ -0.17625268   0.21232972]
 [ -0.17625268   0.21232972]
 [ -0.24166967   0.23852092]
 [ -0.17625268   0.21232972]
 [ -0.17625268   0.21232972]
 [  2.9704013   -5.000061  ]
 [ -0.18189001   0.21357395]
 [ -1.247306     1.0205401 ]
 [ -0.2866191    0.34850958]
 [ -0.9695012    0.86642605]
 [ -0.06732882   0.12254012]
 [  0.04039473  -0.31485456]
 [ -1.5151598    1.6532279 ]
 [  0.3075761   -0.12956661]
 [ -9.588549    -6.570454  ]
 [ -3.2791882    0.81592137]
 [ -0.20185024   0.2338621 ]
 [ -0.18589057   0.17991573]
 [ -0.74248946  -9.380539  ]
 [ -4.483707     4.679197  ]
 [ -1.4918826    1.0383223 ]
 [ -0.35191268   0.2844535 ]
 [-13.356003     1.9265727 ]
 [ -0.17625268   0.21232972]
 [ -1.214593     0.759784  ]
 [ -0.1622526    0.1941756 ]
 [ -0.1762527    0.21232972]]
Accuracy: 0.121212
Precision: 0.000000
Rec

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#MobileVNet

In [ ]:
mobilenetV2 = MobileNetV2(input_shape=(224, 224, 3),
                          include_top=False,
                          weights='imagenet'
                          )

average_layer = GlobalAveragePooling2D()

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
model = Sequential([
    mobilenetV2,
    average_layer,
    Dense(256, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.1),
    Dense(2, activation=tf.nn.softmax)
])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
19/19 [==============================] - 6s 143ms/step - loss: 0.3934 - accuracy: 0.8458 - val_loss: 2.3631 - val_accuracy: 0.0909 - lr: 0.0010
Epoch 2/5
19/19 [==============================] - 1s 43ms/step - loss: 0.1165 - accuracy: 0.9525 - val_loss: 1.6944 - val_accuracy: 0.2424 - lr: 0.0010
Epoch 3/5
19/19 [==============================] - 1s 43ms/step - loss: 0.1061 - accuracy: 0.9525 - val_loss: 1.5061 - val_accuracy: 0.3030 - lr: 0.0010
Epoch 4/5
19/19 [==============================] - 1s 42ms/step - loss: 0.0613 - accuracy: 0.9780 - val_loss: 1.4617 - val_accuracy: 0.3030 - lr: 1.0000e-04
Epoch 5/5
19/19 [==============================] - 1s 42ms/step - loss: 0.0877 - accuracy: 0.9644 - val_loss: 1.4432 - val_accuracy: 0.3333 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 1.4432 - accuracy: 0.3333 - 78ms/epoch - 39ms/step
0.3333333432674408


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.10175401 0.898246  ]
 [0.25007844 0.74992156]
 [0.01067632 0.9893237 ]
 [0.23227468 0.76772535]
 [0.55391973 0.44608024]
 [0.02381363 0.9761864 ]
 [0.9648707  0.03512931]
 [0.30201674 0.6979832 ]
 [0.16579923 0.8342008 ]
 [0.17120025 0.8287997 ]
 [0.24117465 0.75882536]
 [0.5182303  0.48176968]
 [0.02399185 0.9760081 ]
 [0.2368485  0.7631515 ]
 [0.917007   0.0829929 ]
 [0.75809395 0.24190603]
 [0.5267986  0.47320136]
 [0.28984606 0.71015394]
 [0.95485336 0.04514667]
 [0.1432498  0.85675025]
 [0.99838686 0.00161315]
 [0.20515688 0.7948431 ]
 [0.22473587 0.7752642 ]
 [0.41358763 0.5864124 ]
 [0.03870054 0.9612995 ]
 [0.18838246 0.81161755]
 [0.6417273  0.35827264]
 [0.30179086 0.6982091 ]
 [0.6042849  0.39571512]
 [0.75966126 0.2403387 ]
 [0.25733393 0.74266607]
 [0.15815176 0.8418482 ]
 [0.0580897  0.9419103 ]]
Accuracy: 0.333333
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[11 22]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#VGG16

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
vgg16_model.output[-1]

<KerasTensor: shape=(7, 7, 512) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [ ]:
model = Sequential([vgg16_model,
                    Flatten(),
#                     GlobalAveragePooling2D(),
#                     Dense(512, activation = "relu"),
#                     BatchNormalization(),
#                     Dropout(0.3),
#                     Dense(128, activation = "relu"),
#                     Dropout(0.1),
#                     # Dense(32, activation = "relu"),
#                     # Dropout(0.3),
                    Dense(2, activation = "softmax")])

In [ ]:
model.layers[0].trainable = False
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_5 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
19/19 [==============================] - 9s 310ms/step - loss: 1.7819 - accuracy: 0.8983 - val_loss: 10.1828 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 2/5
19/19 [==============================] - 3s 163ms/step - loss: 0.2624 - accuracy: 0.9831 - val_loss: 40.8256 - val_accuracy: 0.4545 - lr: 0.0010
Epoch 3/5
19/19 [==============================] - 3s 164ms/step - loss: 0.1297 - accuracy: 0.9915 - val_loss: 33.9284 - val_accuracy: 0.6061 - lr: 0.0010
Epoch 4/5
19/19 [==============================] - 3s 165ms/step - loss: 2.0018e-04 - accuracy: 1.0000 - val_loss: 32.1954 - val_accuracy: 0.6364 - lr: 1.0000e-04
Epoch 5/5
19/19 [==============================] - 3s 167ms/step - loss: 1.4871e-04 - accuracy: 1.0000 - val_loss: 31.9300 - val_accuracy: 0.6364 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 31.9300 - accuracy: 0.6364 - 180ms/epoch - 90ms/step
0.6363636255264282


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[5.63541925e-32 1.00000000e+00]
 [1.11525664e-02 9.88847375e-01]
 [0.00000000e+00 1.00000000e+00]
 [1.00000000e+00 6.65270509e-22]
 [6.71087128e-22 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [1.00000000e+00 3.54188796e-15]
 [0.00000000e+00 1.00000000e+00]
 [1.56078745e-30 1.00000000e+00]
 [1.00000000e+00 6.39998441e-17]
 [1.00000000e+00 7.15128123e-10]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 3.58157320e-16]
 [9.99977946e-01 2.20433431e-05]
 [9.99998808e-01 1.14114812e-06]
 [1.48600119e-03 9.98513997e-01]
 [1.00000000e+00 1.26349138e-23]
 [1.00000000e+00 1.51545230e-08]
 [1.00000000e+00 2.18402367e-17]
 [1.00000000e+00 6.39097396e-27]
 [1.00000000e+00 5.64231217e-10]
 [1.00000000e+00 8.03880296e-11]
 [1.00000000e+00 4.58240591e-23]
 [1.00000000e+00 1.99447750e-11]
 [9.70942676e-01 2.90573500e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.83974791e-19 1.00000000e+00]
 [1.00000000e+00 4.60408384e-14]
 [9.72731791e-12 1.00000000e+00]
 [1.00000000e+00 1.26271821e-34]
 [2.997685

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Pre-Trained VGG19

In [ ]:
vgg19_model = tf.keras.applications.vgg19.VGG19(include_top=True, weights="imagenet", input_shape=(224,224,3))

574726144/574710816 [==============================] - 3s 0us/step


In [ ]:
vgg19_model.output[-1]

<KerasTensor: shape=(1000,) dtype=float32 (created by layer 'tf.__operators__.getitem_1')>

In [ ]:
model = Sequential()
model.add(vgg19_model)
model.add(Flatten())

model.add(Dense(512, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(128, activation = "relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.1))


model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               512512    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                      

In [ ]:
model.layers[0].trainable = False
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
19/19 [==============================] - 6s 215ms/step - loss: 0.4328 - accuracy: 0.8169 - val_loss: 0.7773 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 4s 202ms/step - loss: 0.1029 - accuracy: 0.9644 - val_loss: 0.9082 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 4s 204ms/step - loss: 0.0777 - accuracy: 0.9814 - val_loss: 0.9890 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 4s 204ms/step - loss: 0.0396 - accuracy: 0.9847 - val_loss: 1.0287 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
19/19 [==============================] - 4s 212ms/step - loss: 0.0351 - accuracy: 0.9915 - val_loss: 1.0768 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 6/10
19/19 [==============================] - 4s 203ms/step - loss: 0.0373 - accuracy: 0.9831 - val_loss: 1.1016 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 7/10
19/19 [================

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 1.1702 - accuracy: 0.0000e+00 - 236ms/epoch - 118ms/step
0.0


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.33527827 0.6647218 ]
 [0.29698053 0.70301944]
 [0.26706466 0.73293537]
 [0.3000736  0.6999264 ]
 [0.29019654 0.70980346]
 [0.24291582 0.7570842 ]
 [0.3217787  0.6782213 ]
 [0.26595718 0.7340429 ]
 [0.30383345 0.6961666 ]
 [0.30867058 0.6913294 ]
 [0.32261786 0.6773821 ]
 [0.33136678 0.6686332 ]
 [0.38032967 0.61967033]
 [0.3140368  0.68596315]
 [0.28915784 0.71084213]
 [0.29281402 0.70718604]
 [0.34131384 0.65868616]
 [0.31349495 0.6865051 ]
 [0.32102138 0.67897856]
 [0.39574066 0.6042594 ]
 [0.3211759  0.67882407]
 [0.25970626 0.74029374]
 [0.3571249  0.64287513]
 [0.38898164 0.61101836]
 [0.2923088  0.7076912 ]
 [0.30518177 0.6948182 ]
 [0.2648532  0.7351468 ]
 [0.31720057 0.68279946]
 [0.30870277 0.69129723]
 [0.38153678 0.6184633 ]
 [0.37598702 0.624013  ]
 [0.31695566 0.6830443 ]
 [0.20957631 0.7904237 ]]
Accuracy: 0.000000
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[ 0 33]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#DenseNet 

In [ ]:
densenet = DenseNet121(weights='imagenet',
                           include_top=False,
                           input_shape=(224,224,3)
                          )

29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.models.Sequential([densenet,
                                        GlobalAveragePooling2D(),
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),
                                        Dense(256, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.2),                                        
                                        Dense(128, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.1),
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_10 (Dense)            (None, 512)               524800    
                                                                 
 batch_normalization_5 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 256)              

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=15, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/15
19/19 [==============================] - 14s 294ms/step - loss: 0.4038 - accuracy: 0.8203 - val_loss: 0.1826 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 2/15
19/19 [==============================] - 2s 103ms/step - loss: 0.2266 - accuracy: 0.9119 - val_loss: 0.8431 - val_accuracy: 0.6364 - lr: 0.0010
Epoch 3/15
19/19 [==============================] - 2s 98ms/step - loss: 0.1565 - accuracy: 0.9441 - val_loss: 1.1129 - val_accuracy: 0.5758 - lr: 0.0010
Epoch 4/15
19/19 [==============================] - 2s 95ms/step - loss: 0.1641 - accuracy: 0.9407 - val_loss: 1.4755 - val_accuracy: 0.4848 - lr: 1.0000e-04
Epoch 5/15
19/19 [==============================] - 2s 98ms/step - loss: 0.1252 - accuracy: 0.9424 - val_loss: 1.6155 - val_accuracy: 0.4848 - lr: 1.0000e-04
Epoch 6/15
19/19 [==============================] - 2s 96ms/step - loss: 0.1457 - accuracy: 0.9424 - val_loss: 1.6931 - val_accuracy: 0.4242 - lr: 1.0000e-04
Epoch 7/15
19/19 [==============================] - 2s 96ms/s

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 1.9598 - accuracy: 0.3939 - 154ms/epoch - 77ms/step
0.39393940567970276


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.00103143 0.9989686 ]
 [0.02650721 0.9734928 ]
 [0.00994898 0.990051  ]
 [0.59902316 0.4009768 ]
 [0.01847462 0.9815254 ]
 [0.00479133 0.9952087 ]
 [0.9971769  0.00282307]
 [0.1741588  0.8258412 ]
 [0.00888512 0.99111485]
 [0.97217166 0.02782827]
 [0.30141997 0.69858   ]
 [0.83521074 0.16478924]
 [0.01142934 0.98857063]
 [0.00438651 0.99561346]
 [0.8674689  0.13253117]
 [0.35877612 0.6412239 ]
 [0.01902425 0.98097575]
 [0.3932091  0.6067909 ]
 [0.54979384 0.45020613]
 [0.17816971 0.82183033]
 [0.99287945 0.00712053]
 [0.6269275  0.37307248]
 [0.9027378  0.09726218]
 [0.8355064  0.16449362]
 [0.04895283 0.9510472 ]
 [0.6668225  0.33317748]
 [0.98714954 0.01285048]
 [0.3855423  0.6144577 ]
 [0.440331   0.559669  ]
 [0.9647445  0.0352555 ]
 [0.12528397 0.87471604]
 [0.3318857  0.6681143 ]
 [0.01275566 0.9872443 ]]
Accuracy: 0.393939
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[13 20]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#inception V3

In [ ]:
inc_model = InceptionV3(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
print("number of layers in the model : ", len(inc_model.layers))

number of layers in the model :  311


In [ ]:
model = tf.keras.models.Sequential([inc_model,
                                        GlobalAveragePooling2D(),
                                    
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),   
                                                                     
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_14 (Dense)            (None, 512)               1049088   
                                                                 
 batch_normalization_102 (Ba  (None, 512)              2048      
 tchNormalization)                                               
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 2)                

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=15, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/15
19/19 [==============================] - 10s 237ms/step - loss: 0.5256 - accuracy: 0.8034 - val_loss: 5.6515 - val_accuracy: 0.4545 - lr: 0.0010
Epoch 2/15
19/19 [==============================] - 2s 84ms/step - loss: 0.2741 - accuracy: 0.8983 - val_loss: 3.5106 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 3/15
19/19 [==============================] - 2s 89ms/step - loss: 0.2377 - accuracy: 0.9017 - val_loss: 1.1075 - val_accuracy: 0.6061 - lr: 0.0010
Epoch 4/15
19/19 [==============================] - 2s 91ms/step - loss: 0.1736 - accuracy: 0.9305 - val_loss: 1.6437 - val_accuracy: 0.3939 - lr: 1.0000e-04
Epoch 5/15
19/19 [==============================] - 2s 87ms/step - loss: 0.1647 - accuracy: 0.9373 - val_loss: 1.9820 - val_accuracy: 0.3030 - lr: 1.0000e-04
Epoch 6/15
19/19 [==============================] - 2s 94ms/step - loss: 0.1587 - accuracy: 0.9373 - val_loss: 1.9342 - val_accuracy: 0.2727 - lr: 1.0000e-04
Epoch 7/15
19/19 [==============================] - 2s 93ms/st

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 1.8514 - accuracy: 0.2727 - 146ms/epoch - 73ms/step
0.27272728085517883


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.17209856 0.8279015 ]
 [0.06259415 0.9374059 ]
 [0.0063809  0.993619  ]
 [0.88913906 0.11086098]
 [0.07908712 0.92091286]
 [0.01236628 0.9876337 ]
 [0.58151156 0.41848847]
 [0.00920286 0.99079716]
 [0.30163613 0.6983639 ]
 [0.84427387 0.15572616]
 [0.20845143 0.7915485 ]
 [0.8905715  0.10942853]
 [0.28725025 0.7127497 ]
 [0.34542474 0.6545752 ]
 [0.3122589  0.6877411 ]
 [0.15411822 0.84588176]
 [0.16029836 0.8397016 ]
 [0.8851782  0.11482181]
 [0.24222876 0.75777125]
 [0.09196458 0.9080354 ]
 [0.4306478  0.5693522 ]
 [0.82104695 0.17895305]
 [0.03362539 0.9663746 ]
 [0.05946315 0.94053686]
 [0.23008586 0.76991415]
 [0.29149824 0.7085017 ]
 [0.09175579 0.9082442 ]
 [0.01231764 0.9876823 ]
 [0.8855053  0.11449472]
 [0.82447535 0.17552459]
 [0.74414355 0.2558565 ]
 [0.03515619 0.9648438 ]
 [0.01252054 0.9874794 ]]
Accuracy: 0.272727
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[ 9 24]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#ResNet 50

In [ ]:
res = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.models.Sequential([res,
                                        GlobalAveragePooling2D(),
                                    
                                        Dense(512, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.3),   
                                    
                                        Dense(256, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.2),  
                                    
                                        Dense(128, activation='relu'),
                                        BatchNormalization(),
                                        Dropout(0.1),  
                                                                     
                                        Dense(2, activation='softmax')
                                      ])

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_16 (Dense)            (None, 512)               1049088   
                                                                 
 batch_normalization_103 (Ba  (None, 512)              2048      
 tchNormalization)                                               
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 256)              

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=5, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/5
19/19 [==============================] - 9s 221ms/step - loss: 0.1973 - accuracy: 0.9203 - val_loss: 3.0783 - val_accuracy: 0.4242 - lr: 0.0010
Epoch 2/5
19/19 [==============================] - 2s 99ms/step - loss: 0.0590 - accuracy: 0.9763 - val_loss: 1.8770 - val_accuracy: 0.6061 - lr: 0.0010
Epoch 3/5
19/19 [==============================] - 2s 105ms/step - loss: 0.0344 - accuracy: 0.9898 - val_loss: 5.0262 - val_accuracy: 0.3636 - lr: 0.0010
Epoch 4/5
19/19 [==============================] - 2s 105ms/step - loss: 0.0152 - accuracy: 0.9932 - val_loss: 4.1889 - val_accuracy: 0.3939 - lr: 1.0000e-04
Epoch 5/5
19/19 [==============================] - 2s 103ms/step - loss: 0.0193 - accuracy: 0.9932 - val_loss: 3.5281 - val_accuracy: 0.4848 - lr: 1.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 3.5281 - accuracy: 0.4848 - 189ms/epoch - 94ms/step
0.4848484992980957


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[5.8697069e-06 9.9999416e-01]
 [9.1203040e-04 9.9908793e-01]
 [3.7013961e-06 9.9999630e-01]
 [9.9994922e-01 5.0768555e-05]
 [1.2944602e-02 9.8705542e-01]
 [1.3077825e-05 9.9998689e-01]
 [9.9025601e-01 9.7440565e-03]
 [3.4387188e-04 9.9965608e-01]
 [2.7286782e-04 9.9972707e-01]
 [2.0406976e-01 7.9593021e-01]
 [8.2143253e-01 1.7856747e-01]
 [9.9971658e-01 2.8344497e-04]
 [1.8255814e-03 9.9817443e-01]
 [5.3819686e-01 4.6180314e-01]
 [9.9650288e-01 3.4970732e-03]
 [8.0242520e-03 9.9197572e-01]
 [1.5358465e-03 9.9846423e-01]
 [6.5040737e-02 9.3495929e-01]
 [9.9212390e-01 7.8760386e-03]
 [9.9707627e-01 2.9237890e-03]
 [9.9994695e-01 5.2997693e-05]
 [9.9982113e-01 1.7892507e-04]
 [9.9650747e-01 3.4924534e-03]
 [9.9631232e-01 3.6876318e-03]
 [5.1334453e-01 4.8665547e-01]
 [9.4399428e-01 5.6005739e-02]
 [1.2555179e-02 9.8744476e-01]
 [3.6427997e-02 9.6357197e-01]
 [2.5050176e-02 9.7494984e-01]
 [9.9453461e-01 5.4653753e-03]
 [5.4555786e-01 4.5444217e-01]
 [3.4061322e-01 6.5938681e-01]
 [6.6054

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#SSD

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_97 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 conv2d_98 (Conv2D)          (None, 222, 222, 64)      18496     
                                                                 
 conv2d_99 (Conv2D)          (None, 220, 220, 128)     73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 110, 110, 128)    0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 110, 110, 128)     0         
                                                                 
 flatten_3 (Flatten)         (None, 1548800)           0         
                                                      

In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
19/19 [==============================] - 12s 407ms/step - loss: 808.7234 - accuracy: 0.5237 - val_loss: 45.0748 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 5s 288ms/step - loss: 3.8860 - accuracy: 0.5678 - val_loss: 0.6991 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 6s 298ms/step - loss: 0.6884 - accuracy: 0.5661 - val_loss: 0.7046 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 5s 284ms/step - loss: 0.6875 - accuracy: 0.5644 - val_loss: 0.7051 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
19/19 [==============================] - 5s 284ms/step - loss: 0.6861 - accuracy: 0.5661 - val_loss: 0.7058 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 6/10
19/19 [==============================] - 5s 283ms/step - loss: 0.6871 - accuracy: 0.5644 - val_loss: 0.7065 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 7/10
19/19 [============

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.7094 - accuracy: 0.0000e+00 - 151ms/epoch - 76ms/step
0.0


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]
 [0.4919452  0.50805473]]
Accuracy: 0.000000
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[ 0 33]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#AlexNet

In [ ]:
model=Sequential()

#1 conv layer
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",activation="relu",input_shape=(224,224,3)))

#1 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#2 conv layer
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"))

#2 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())

#3 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#4 conv layer
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#5 conv layer
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))

#3 max pool layer
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(BatchNormalization())


model.add(Flatten())

#1 dense layer
model.add(Dense(4096,input_shape=(227,227,3),activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#2 dense layer
model.add(Dense(4096,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#3 dense layer
model.add(Dense(1000,activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

#output layer
model.add(Dense(2,activation="softmax"))

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=10, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/10
19/19 [==============================] - 5s 115ms/step - loss: 1.3622 - accuracy: 0.6610 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 1s 38ms/step - loss: 0.6414 - accuracy: 0.7508 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 1s 35ms/step - loss: 0.4809 - accuracy: 0.8051 - val_loss: 3.6124e-09 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 1s 31ms/step - loss: 0.4697 - accuracy: 0.8203 - val_loss: 0.0658 - val_accuracy: 0.9394 - lr: 1.0000e-04
Epoch 5/10
19/19 [==============================] - 1s 30ms/step - loss: 0.2849 - accuracy: 0.8847 - val_loss: 0.3693 - val_accuracy: 0.8788 - lr: 1.0000e-04
Epoch 6/10
19/19 [==============================] - 1s 31ms/step - loss: 0.2812 - accuracy: 0.9000 - val_loss: 0.5222 - val_accuracy: 0.8485 - lr: 1.0000e-04
Epoch 7/10
19/19 [==============================] -

In [ ]:
test_loss, test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 0.6834 - accuracy: 0.6061 - 34ms/epoch - 17ms/step
0.6060606241226196


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs,axis=1) #1-D array of classes
yhat_probs = yhat_probs[:, 0] # reduce to 1d array


accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[[0.18254788 0.81745213]
 [0.18804505 0.811955  ]
 [0.24345255 0.75654745]
 [0.8284083  0.17159173]
 [0.41643992 0.58356005]
 [0.0681258  0.9318742 ]
 [0.9577556  0.04224439]
 [0.4509845  0.5490155 ]
 [0.14837134 0.85162866]
 [0.6343261  0.3656739 ]
 [0.5485587  0.45144132]
 [0.9617304  0.03826957]
 [0.67185956 0.32814047]
 [0.3725043  0.62749565]
 [0.73291755 0.26708245]
 [0.5485896  0.4514104 ]
 [0.86012006 0.13988   ]
 [0.7286327  0.27136737]
 [0.44934705 0.5506529 ]
 [0.7756132  0.22438681]
 [0.9986527  0.00134727]
 [0.9782833  0.02171666]
 [0.81698394 0.18301605]
 [0.86333764 0.13666236]
 [0.91751105 0.082489  ]
 [0.45031267 0.5496873 ]
 [0.3982793  0.60172063]
 [0.7826871  0.21731289]
 [0.88610375 0.11389625]
 [0.9290856  0.0709144 ]
 [0.30105886 0.6989411 ]
 [0.65145075 0.34854928]
 [0.12111323 0.8788868 ]]
Accuracy: 0.606061
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
[[20 13]
 [ 0  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#GoogleNet

In [ ]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(5, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(1000, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(2, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
model = GoogLeNet()

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy']
                )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
hist = model.fit(x=tr_images, y = tr_labels, 
                    epochs=20, validation_data=(ts_images, ts_labels),
                    callbacks=[lr_callbacks])

Epoch 1/20
19/19 [==============================] - 14s 313ms/step - loss: 5.8562 - dense_30_loss: 1.3898 - dense_27_loss: 2.4101 - dense_29_loss: 2.0563 - dense_30_accuracy: 0.5136 - dense_27_accuracy: 0.5390 - dense_29_accuracy: 0.4797 - val_loss: 2.3831 - val_dense_30_loss: 0.6884 - val_dense_27_loss: 1.1563 - val_dense_29_loss: 0.5385 - val_dense_30_accuracy: 0.4545 - val_dense_27_accuracy: 0.3333 - val_dense_29_accuracy: 1.0000 - lr: 0.0010
Epoch 2/20
19/19 [==============================] - 2s 107ms/step - loss: 2.0258 - dense_30_loss: 0.5988 - dense_27_loss: 0.6473 - dense_29_loss: 0.7797 - dense_30_accuracy: 0.6847 - dense_27_accuracy: 0.6966 - dense_29_accuracy: 0.6441 - val_loss: 2.2967 - val_dense_30_loss: 0.6607 - val_dense_27_loss: 0.8435 - val_dense_29_loss: 0.7925 - val_dense_30_accuracy: 0.4545 - val_dense_27_accuracy: 0.4242 - val_dense_29_accuracy: 0.4545 - lr: 0.0010
Epoch 3/20
19/19 [==============================] - 2s 106ms/step - loss: 1.3962 - dense_30_loss: 0.4

In [ ]:
test_acc = model.evaluate(ts_images,  ts_labels, verbose=2)
print(test_acc)

2/2 - 0s - loss: 5.1237 - dense_30_loss: 1.6799 - dense_27_loss: 1.6883 - dense_29_loss: 1.7556 - dense_30_accuracy: 0.4545 - dense_27_accuracy: 0.4242 - dense_29_accuracy: 0.4545 - 85ms/epoch - 42ms/step
[5.123749732971191, 1.6798522472381592, 1.6883286237716675, 1.7555689811706543, 0.4545454680919647, 0.42424243688583374, 0.4545454680919647]


In [ ]:
yhat_probs = model.predict(ts_images, verbose=0) # predict probabilities for test set
print(yhat_probs)
yhat_classes = np.argmax(yhat_probs[0],axis=1) #1-D array of classes

accuracy = accuracy_score(ts_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
precision = precision_score(ts_labels, yhat_classes)
print('Precision: %f' % precision)
recall = recall_score(ts_labels, yhat_classes)
print('Recall: %f' % recall)
f1 = f1_score(ts_labels, yhat_classes)
print('F1 score: %f' % f1)

matrix = confusion_matrix(ts_labels, yhat_classes)
print(matrix)

[array([[2.17124559e-02, 9.78287518e-01],
       [1.86180342e-02, 9.81382012e-01],
       [2.37147398e-02, 9.76285279e-01],
       [9.44728494e-01, 5.52715212e-02],
       [2.71234680e-02, 9.72876489e-01],
       [2.03641187e-02, 9.79635835e-01],
       [9.82151031e-01, 1.78489890e-02],
       [3.69138092e-01, 6.30861878e-01],
       [1.53063405e-02, 9.84693646e-01],
       [3.18431050e-01, 6.81568980e-01],
       [2.21383914e-01, 7.78616071e-01],
       [7.73437619e-01, 2.26562336e-01],
       [1.36352060e-02, 9.86364722e-01],
       [3.52528766e-02, 9.64747071e-01],
       [9.59993541e-01, 4.00064923e-02],
       [3.75932455e-01, 6.24067485e-01],
       [1.78731065e-02, 9.82126892e-01],
       [9.68399644e-01, 3.16003449e-02],
       [9.18141484e-01, 8.18584710e-02],
       [7.97953427e-01, 2.02046648e-01],
       [9.97325778e-01, 2.67425226e-03],
       [9.90595937e-01, 9.40412469e-03],
       [7.87285686e-01, 2.12714270e-01],
       [5.25080621e-01, 4.74919349e-01],
       [4.75146

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
